In [36]:
#Initial imports
import pandas as pd
import hvplot.pandas
import panel as pn
import holoviews as hv
#hv.extention('bokeh')
#pn.extention('tabulator')
PALETTE = ["#00204F", "#510063", "#9C8800"]
import alpaca_trade_api as tradeapi
import random as random
#Ignore verbose warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [21]:
#Function to simulate weights for the stocks to use in portfolio construction. It takes the list of tickers as input and the # of combinations the user would like to use
def simulate_weights(tickers, no_of_trials):
    numOfAssets=len(tickers)
    trials = no_of_trials
    overall_weights=[]
    total = 0
    for i in range(trials):
        weights=[]
        for i in range(numOfAssets):
            weights.append(random.random())
        total = sum(weights)
        for i in range(len(weights)):
            weights[i]=round(weights[i]/total,2)
        overall_weights.append(weights)
    #return weights list
    return(overall_weights)

In [22]:
#Function to calculate the average return and the std_deviation of the stock prices for use in portfolio contruction.
def calculate_stock_metrics(tickers):
    start_date = '2020-01-01'
    end_date = '2020-12-31'
    timeframe = '1D'
    alpaca_api_key = 'PKURMS5ELMEHV4ZGBXTQ'
    alpaca_secret_key = 'NhYNWSU4szskB7qUMl2FXPqYoQZXwL50ahf54jds'
    # Create the Alpaca API object
    alpaca = tradeapi.REST(
        alpaca_api_key,
        alpaca_secret_key,
        api_version="v2")
    # Get current price data for tickers
    alpaca_df = alpaca.get_bars(
        tickers,
        timeframe,
        start = start_date,
        end = end_date
    ).df
    #Clean the stock data and store in a separate dataframe for analysis
    alpaca_df = alpaca_df.drop(columns=['vwap','trade_count','open', 'high', 'low', 'volume'])
    stock_df = pd.DataFrame(columns=['ticker','mean_close','mean_return'])#,'std_dev'])
    counter = 0
    for ticker in tickers:
        #record the stock in stock_df
        stock_df.at[counter, 'ticker'] = ticker
        #calculate the mean price for the ticker
        stock_df.at[counter,'mean_return'] = alpaca_df.groupby('symbol').get_group(ticker)['close'].pct_change().mean()
        #calculate the mean daily return for the ticker
        stock_df.at[counter,'mean_close'] = alpaca_df.groupby('symbol').get_group(ticker)['close'].mean()
        #calculate the standard deviation of the daily return for the ticker
        #stock_df.at[counter, 'std_dev'] = alpaca_df.groupby('symbol').get_group(ticker)['close'].pct_change().std()
        counter += 1
    return(alpaca_df)
    #return(stock_df)

In [23]:
#hardcoded variables to be replaced by user input and environment variable calls in the future
portfolio_stocks = ['MSFT','GOOG', 'AAPL', 'AMZN', 'TSLA']
no_of_trials = 1
portfolio_weights = simulate_weights(portfolio_stocks,no_of_trials)
stock_df = calculate_stock_metrics(portfolio_stocks)

In [29]:
idf = stock_df.interactive()


In [32]:
# Define Panel widgets
cylinders = pn.widgets.IntSlider(name='Symbol', start=4, end=8, step=2)
mfr = pn.widgets.ToggleGroup(
    name='Symbol',
    options=portfolio_stocks, 
    value=portfolio_stocks,
    button_type='success')
yaxis = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['close', 'open'],
    button_type='success'
)


In [34]:
# Combine pipeline and widgets
ipipeline = (
    idf[
        (idf.symbol == cylinders) & 
        (idf.symbol.isin(mfr))
    ]
    .groupby(['symbol', 'timestamp'])[yaxis].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='timestamp')  
    .reset_index(drop=True)
)

In [38]:
ivhplot = ipipeline.hvplot(x='timestamp', y=yaxis, by='symbol', color=PALETTE, line_width=6)
ivhplot

SkipRendering: bokeh backend could not plot any Elements in the Overlay.